In [3]:
!pip install openpyxl

In [4]:
import openpyxl

In [5]:
import xlwings as xw

# Tạo và mở file Excel

In [6]:
from openpyxl import Workbook
# Bước 1: Tạo workbook mới
wb = Workbook()
# Bước 2: Lấy sheet đầu tiên mặc định 
ws = wb.active
# Bước 3: Đổi tên sheet
ws.title = "Bảng Lương"
# Bước 4: Ghi dữ liệu đầu tiên vào ô A1
ws["A1"] = "Lương nhân viên"
# Bước 5: Lưu file lại
wb.save("luong.xlsx")

## Thêm, sửa và xóa giá trị ô

### Cho 1 ô

In [7]:
ws['A1'] = 'Xin Chào'

In [8]:
ws.cell(row=1, column=1).value = 'Xin Chào!'

In [9]:
ws['A1'].value = None

In [10]:
ws.cell(row=1, column=1).value = None

### Cho nhiều ô

In [11]:
ws.append(['ABC', 20, '2025'])
ws.append(['DEF', 30, '2025'])

In [12]:
data = [
    ['ABC', 20, '2025'],
    ['DEF', 30, '2025']
]

for row_index, row_data in enumerate(data, start=1):
    for col_index, value in enumerate(row_data, start=1):
        ws.cell(row=row_index, column=col_index, value=value)

In [13]:
for row in ws["A1:C2"]:
    for cell in row:
        cell.value = None

In [14]:
for row in ws["A1:D2"]:
    for cell in row:
        print(cell.coordinate, cell.value)

A1 None
B1 None
C1 None
D1 None
A2 None
B2 None
C2 None
D2 None


## Mở file excel và thêm nội dung

In [15]:
from openpyxl import load_workbook
# Bước 1: Mở file 
wb = load_workbook("luong.xlsx")
# Bước 2: Chọn sheet làm việc
# ws = wb.active  # mặc định
ws = wb["Bảng Lương"]  #chọn sheet theo tên
# Bước 3: Ghi đè nội dung
ws["A1"] = "Lương nhân viên"
ws.append(['Tên', 'Số giờ làm', 'Lương 1 giờ', 'Lương'])
ws.append(['Nguyễn Văn A', 30, 20000])
ws.append(['Lê Thị B', 35, 21000])
ws.append(['Trần C', 28, 20500])
ws.append(['Nguyễn Văn D', 35, 25000])
# Bước 4: Lưu lại (ghi đè)
wb.save("luong.xlsx")

# Định dạng và căn chỉnh 

In [16]:
ws.merge_cells("A1:D1")
# ws.unmerge_cells("A1:D1")
wb.save("luong.xlsx")

In [17]:
from openpyxl.styles import Alignment

alignment = Alignment(
    horizontal='center', 
    vertical='center', 
    wrap_text=True,
    text_rotation=0,
    shrink_to_fit=False,
    indent=0
)

ws = wb["Bảng Lương"]
ws['A1'].alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Căn giữa ngang và dọc cho nhiều ô 
for row in ws.iter_rows(min_row=2, max_row=2, min_col=1, max_col=4):
    for cell in row:
        cell.alignment = Alignment(horizontal="center", vertical="center")

wb.save("luong.xlsx")

In [18]:
from openpyxl.styles import Font

ws["A1"].font = Font(size=14, bold=True, color="000000")
wb.save("luong.xlsx")

In [19]:
from openpyxl.styles import PatternFill

ws["A1"].fill = PatternFill(fill_type="solid", start_color="FFFFFF")
wb.save("luong.xlsx")

In [20]:
from openpyxl.styles import Border, Side

max_row = ws.max_row
max_col = ws.max_column

thin = Side(border_style="thin", color="000000")
border = Border(left=thin, right=thin, top=thin, bottom=thin)

# Áp dụng border cho từng ô trong vùng
for row in ws.iter_rows(min_row=1, max_row=max_row, min_col=0, max_col=max_col):
    for cell in row:
        cell.border = border

wb.save("luong.xlsx")

In [21]:
# Định dạng cột C và D (cột Lương 1 giờ và Lương ) thành tiền VND
for row in ws.iter_rows(min_row=3, min_col=3, max_col=4):
    for cell in row:
        cell.number_format = '#,##0" VND"'  # Ký hiệu tiền Việt Nam
wb.save("luong.xlsx")

In [22]:
from openpyxl.utils import get_column_letter

# Tìm cột có tiêu đề là "Lương 1 giờ" hoặc "Lương nhận" trong hàng 2
header_row = 2
target_headers = ["Lương 1 giờ", "Lương"]

col_indexes = {}  # Từ điển lưu tên cột -> chỉ số

for col in range(1, ws.max_column + 1):
    cell_value = ws.cell(row=header_row, column=col).value
    if cell_value in target_headers:
        col_indexes[cell_value] = col

# Định dạng các cột theo tên đã tìm được
for header, col in col_indexes.items():
    col_letter = get_column_letter(col)
    for row in range(3, ws.max_row + 1):  # Bỏ qua tiêu đề
        ws[f"{col_letter}{row}"].number_format = '#,##0" VND"'

# Tính toán các giá trị và gọi hàm

## Tính trực tiếp trong Python

In [23]:
# Duyệt từng dòng để tính Lương = Lương 1 giờ * Số giờ làm 
for row in ws.iter_rows(min_row=3, min_col=2, max_col=3):
    gio_lam = row[0].value
    luong_1_gio = row[1].value
    luong = gio_lam * luong_1_gio if gio_lam and luong_1_gio else 0
    ws.cell(row=row[0].row, column=4).value = luong

# Lưu lại
wb.save("luong_python_updated.xlsx")

In [24]:
max_row = ws.max_row
for i in range(3, max_row+1):
    ws[f"D{i}"] = f"=B{i}*C{i}"

# Lưu lại 
wb.save("luong_excel_updated.xlsx")
wb.save("luong.xlsx")

## Biểu đồ

In [25]:
from openpyxl.chart import BarChart, Reference

# Bước 1: Tạo biểu đồ cột
chart = BarChart()
chart.title = "Lương nhân viên"
chart.x_axis.title = "Nhân viên"
chart.y_axis.title = "Lương (VND)"

# Bước 2: Xác định vùng dữ liệu và nhãn 
data = Reference(ws, min_col=4, min_row=2, max_row=max_row)
categories = Reference(ws, min_col=1, min_row=3, max_row=max_row)

# Bước 3: Gán dữ liệu cho biểu đồ
chart.add_data(data, titles_from_data=True)
chart.set_categories(categories)

# Bước 4: Chèn biểu đồ
ws.add_chart(chart, "F2")

# Lưu lại
wb.save("luong.xlsx")